#  Coursara Capstone  - Toronto Neighborhoods

## This project will compare neighborhoods in the Toronto area

By Brian Hoyler

## Part 1

create datatable with Toronto Neighborhoods

In [10]:
import pandas as pd
import numpy as nb
print('Libraries imported.')

Libraries imported.


### Install the requirements for BeautifulSoup

In [15]:
#!pip install bs4
#!pip install lxml
#!pip install html5lib
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
print("BeautifulSoup installed")

BeautifulSoup installed


### define the URL for the data and use BeautifulSoup to scrape the data

In [16]:
#The below url contains html tables with data about Toronto Neighborhoods.
url = "https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto"
url2 = "https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods"
url3 = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [17]:
# get the contents of the webpage in text format and store in a variable called data
data  = requests.get(url3).text

In [18]:
#soup = BeautifulSoup(data,"lxml")
soup = BeautifulSoup(data,"html5lib")

In [19]:
#find all html tables in the web page
tables = soup.find_all('table') # in html table is represented by the tag <table>

### The following code produces the columns for the dataframe and drops rows with unassigned boroughs

In [20]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
#print(table_contents)

### build dataframe and modify some of the borough names

In [21]:
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

## Results of Step One - Creating a dataframe of postalcodes, Boroughs and Neighborhoods

In [22]:
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [23]:
df.shape

(103, 3)

## First portion is complete.  The dataframe has 103 rows.

# Beginning of the second section

### define the geocoder function

In [7]:
import geocoder # import geocoder

def get_coordinates():
# initialize your variable to None
    lat_lng_coords = None

# loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format('M5A'))
        lat_lng_coords = g.latlng
    return lat_lng_coords



### determine the latitude and longitude for each postal code

In [37]:
#lat_lng_coords = get_coordinates()
#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]
#print(latitude,longitude)

### read the csv file

In [13]:
latlongdf = pd.read_csv('Geospatial_Coordinates.csv')

In [36]:
latlongdf.rename(columns = {'Postal Code':'PostalCode'}, inplace = True) 


### now merge the postal code dataframe with the latitude and longitude dataframe.

In [31]:
mergeddf = pd.merge(df, latlongdf, on="PostalCode")

In [32]:
mergeddf

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


In [35]:
mergeddf.shape

(103, 5)

## Shape of complete data frame is 103 rows by 5 columns

# End of the Second Section